In [13]:
import numpy as np
import pandas as pd
import numpy.random as rnd
from scipy.special import expit
import seaborn as sns
from tqdm.notebook import tqdm

from IPython.display import Markdown, display

# Evaluation of our DP mechanisms

In [14]:
# Variances of the estimator of the performance gap for each mechanism

# Variance for M_L mechanism
def varVL(K, g, e, ν2=1):
    '''
    K is the total number of clients.
    g = n / K-n
    e is epsilon (overall dp budget)
    ν2 = 1  # to get an upper-bound of the variance of mechanism L
    '''
    def varL(k, e):
        return 2 * ((k / e)**2)

    if e < 2 / 3:
        e1 = np.log(3) - (e / 2)
        vk = varL(2 / 3, e)
        
    elif e >= 2 / 3:
        e1 = np.log(2 / e) + e - 1
        vk = 2
    el = np.exp(-e1)       
       
    g0 = (1 + g) / g
    g1 = g0 * (1 + g)
    g2 = g0 * (1 + g**3) / g
    
    term1 = el*ν2 + varL(2, e)*(1 + el)
    term2 = vk * (el + el*el)
    
    return (1 / K) * (g1 * term1 + g2 * term2)
varVL_vect = np.vectorize(varVL)


# Variance for M_R mechanism
def varVR(K, g, e, ν2=0):
    '''
    K is the total number of clients.
    g = n / K-n
    beta = alpha = e (epsilon, overall dp budget)
    ν2 = 0  # to get an upper-bound of the variance of mechanism L
    '''
    b = np.exp(e) / (1 + np.exp(e))
    b1 = 2 * b - 1
    
    a = b
    a1 = (1 - a) / a
    
    g0 = (1 + g) / g
    g1 = g0 * (1 + g)
    g2 = g0 * (1 + g**3) / g
    
    d = a * b1 * b1
    term1 = (1 - ν2 * a * b1 * b1) / d
    term2 = ((1 - a) / a) / d
    
    return (1 / K) * (g1 * term1 + g2 * term2)
varVR_vect = np.vectorize(varVR)

In [15]:
def opt_eps(eps):
    if eps < 2 / 3:
        return 2 / 3, np.log(3) - (eps / 2)
        
    elif 2 >= eps >= 2 / 3:
        return eps, np.log(2 / eps) + eps - 1
    
    elif eps > 2:
        return 2, np.log(2 / eps) + eps - 1
    

def mech_R(e, g, v):
    p = expit(e)
    
    new_g_r, new_v_r = g.copy(), v.copy()
    
    ## protect group
    rr_group_r = rnd.binomial(1, 1 - p, size=g.shape).astype(bool)
    new_g_r[rr_group_r] = 1 - new_g_r[rr_group_r]
    
    ## protect value
    new_v_r[rr_group_r] = 0.
    
    ### Mr
    new_v_r = rnd.binomial(1, (1 + new_v_r) / 2)
    rr_value = rnd.binomial(1, 1 - p, size=v.shape).astype(bool)
    new_v_r[rr_value] = 1 - new_v_r[rr_value]
    new_v_r = 2 * new_v_r - 1
    
    # calculate estimates
    debias_r = n * p * ((2 * p) - 1)
    fnr0_r  = (np.ma.array(new_v_r, mask=new_g_r).sum(1) / debias_r + 1) / 2
    fnr1_r  = (np.ma.array(new_v_r, mask=1 - new_g_r).sum(1) / debias_r + 1) / 2
    
    return fnr0_r, fnr1_r


def mech_L(e, g, v):
    k, e1 = opt_eps(e)
    p1 = expit(e1)

    new_g_l, new_v_l = g.copy(), v.copy()
    
    ## protect group
    rr_group_l = rnd.binomial(1, 1 - p1, size=g.shape).astype(bool)
    new_g_l[rr_group_l] = 1 - new_g_l[rr_group_l]

    ## protect value
    new_v_l[rr_group_l] = 0.    

    ## Ml
    new_v_l[rr_group_l] += rnd.laplace(loc=0.0, scale=2. / e, size=rr_group_l.astype(int).sum())
    new_v_l[~rr_group_l] += rnd.laplace(loc=0.0, scale=k / e, size=(~rr_group_l).astype(int).sum())
    
    # calculate estimates
    debias_l = n * p1
    fnr0_l  = (np.ma.array(new_v_l, mask=new_g_l).sum(1) / debias_l + 1) / 2
    fnr1_l  = (np.ma.array(new_v_l, mask=1 - new_g_l).sum(1) / debias_l + 1) / 2
    
    return fnr0_l, fnr1_l

In [16]:
data = pd.read_csv('data.csv')

epsilons = [0.01, 0.1, 1, 1.78, 10]

n = 5000000
samples = 10
arr = pd.concat([data[data.sex == 0].sample(n),
                 data[data.sex == 1].sample(n)])

v = np.tile(arr.fnr, (samples, 1))
g = np.tile(arr.sex.astype(int), (samples, 1))
v_ = 2 * v - 1

theo0, theo1 = np.ma.array(v, mask=g).mean(1), np.ma.array(v, mask=1 - g).mean(1)
true_gap = np.abs(theo0 - theo1)

In [17]:
errors_r = []
for e in epsilons:
        fnr0_r, fnr1_r = mech_R(e, g, v_)
        
        gap_r = np.abs(fnr0_r - fnr1_r)
        error_gap_r = np.abs(true_gap - gap_r) 
        
        error0_r, error1_r = np.abs(fnr0_r - theo0), np.abs(fnr1_r - theo1)

        error_r = np.abs(np.abs(fnr0_r - fnr1_r) - np.abs(theo0 - theo1))
        
        # 99% prob Chebyshev bound
        var_r = varVR(2*n, 1, e, ν2=0) / 4    # the variance is divided by 4 because we apply the transformation (X+1)/2  to the perf gap, and Var[(X+1)/2] = Var[X]/4
        std_r = 10 * np.sqrt(var_r)  # we multiply by M=10 to obtain a 0.99-probability bound
        
        df = pd.DataFrame({'theo0': theo0,
                           'fnr0': fnr0_r,
                           'error0': error0_r, 
                           'theo1': theo1,
                           'fnr1': fnr1_r,
                           'error1': error1_r,
                           'true_gap': true_gap,
                           'gap': gap_r,
                           'variance_gap': var_r,
                           'error_gap': error_gap_r,
                           'Chebyshev': std_r, 
                          })
        df['e'] = e
        errors_r.append(df)
df_r = pd.concat(errors_r)

In [18]:
df_means_r = df_r.groupby('e').mean()
df_means_r

,theo0,fnr0,error0,theo1,fnr1,error1,true_gap,gap,variance_gap,error_gap,Chebyshev
e,,,,,,,,,,,
0.01,0.00167,0.006006,0.053241,0.213825,0.222891,0.060621,0.212155,0.216885,1.584146e-02,0.073935,1.258629
0.10,0.00167,0.004814,0.008870,0.213825,0.210443,0.006808,0.212155,0.205629,1.453782e-04,0.011653,0.120573
1.00,0.00167,0.001790,0.000426,0.213825,0.214082,0.000418,0.212155,0.212292,8.761762e-07,0.000543,0.009360
1.78,0.00167,0.001766,0.000276,0.213825,0.213888,0.000154,0.212155,0.212121,2.698610e-07,0.000333,0.005195
10.00,0.00167,0.001662,0.000019,0.213825,0.213838,0.000058,0.212155,0.212176,1.000272e-07,0.000063,0.003163


In [19]:
df_std_r = df_r.groupby('e').std()
df_std_r

,theo0,fnr0,error0,theo1,fnr1,error1,true_gap,gap,variance_gap,error_gap,Chebyshev
e,,,,,,,,,,,
0.01,0.0,0.064931,0.032976,0.0,0.079698,0.048578,0.0,0.105795,0.0,0.071720,0.0
0.10,0.0,0.010425,0.005678,0.0,0.008032,0.005073,0.0,0.012567,0.0,0.007373,0.0
1.00,0.0,0.000544,0.000332,0.0,0.000537,0.000409,0.0,0.000640,0.0,0.000321,0.0
1.78,0.0,0.000327,0.000180,0.0,0.000207,0.000144,0.0,0.000476,0.0,0.000323,0.0
10.00,0.0,0.000020,0.000009,0.0,0.000089,0.000066,0.0,0.000086,0.0,0.000059,0.0


In [20]:
errors_l = []
for e in epsilons:
        fnr0_l, fnr1_l = mech_L(e, g, v_)
        
        gap_l = np.abs(fnr0_l - fnr1_l)
        error_gap_l = np.abs(true_gap - gap_l)
        
        error0_l, error1_l = np.abs(fnr0_l - theo0), np.abs(fnr1_l - theo0)
        
        error_l = np.abs(np.abs(fnr0_l - fnr1_l) - np.abs(theo0 - theo1))
        
        # 99% prob Chebyshev bound
        var_l = varVL(2*n, 1, e, ν2=1) / 4   # the variance is divided by 4 because we apply the transformation (X+1)/2 to the perf gap, and Var[(X+1)/2] = Var[X]/4
        std_l = 10 * np.sqrt(var_l)
        
        df = pd.DataFrame({'theo0': theo0,
                           'fnr0': fnr0_l,
                           'error0': error0_l, 
                           'theo1': theo1,
                           'fnr1': fnr1_l,
                           'error1': error1_l,
                           'true_gap': true_gap,
                           'gap': gap_l,
                           'variance_gap': var_l,
                           'error_gap': error_gap_l,
                           'Chebyshev': std_l, 
                          })
        df['e'] = e
        errors_l.append(df)
df_l = pd.concat(errors_l)

In [21]:
df_means_l = df_l.groupby('e').mean()
df_means_l

,theo0,fnr0,error0,theo1,fnr1,error1,true_gap,gap,variance_gap,error_gap,Chebyshev
e,,,,,,,,,,,
0.01,0.00167,0.022291,0.044338,0.213825,0.259754,0.258083,0.212155,0.237463,1.107761e-02,0.043829,1.052502
0.10,0.00167,-0.000998,0.003497,0.213825,0.215290,0.213620,0.212155,0.216288,1.122753e-04,0.008190,0.105960
1.00,0.00167,0.001456,0.000594,0.213825,0.214083,0.212412,0.212155,0.212627,1.400000e-06,0.001001,0.011832
1.78,0.00167,0.001744,0.000379,0.213825,0.213913,0.212243,0.212155,0.212169,5.111901e-07,0.000399,0.007150
10.00,0.00167,0.001686,0.000051,0.213825,0.213820,0.212150,0.212155,0.212134,8.190127e-09,0.000066,0.000905


In [22]:
df_std_l = df_l.groupby('e').std()
df_std_l

,theo0,fnr0,error0,theo1,fnr1,error1,true_gap,gap,variance_gap,error_gap,Chebyshev
e,,,,,,,,,,,
0.01,0.0,0.057870,0.040461,0.0,0.038760,0.038760,0.0,0.061661,0.0,0.048778,0.0
0.10,0.0,0.004238,0.003505,0.0,0.005303,0.005303,0.0,0.008600,0.0,0.004292,0.0
1.00,0.0,0.000725,0.000429,0.0,0.000769,0.000769,0.0,0.001200,0.0,0.000758,0.0
1.78,0.0,0.000487,0.000289,0.0,0.000285,0.000285,0.0,0.000559,0.0,0.000368,0.0
10.00,0.0,0.000057,0.000026,0.0,0.000047,0.000047,0.0,0.000076,0.0,0.000037,0.0


In [23]:
errors = pd.DataFrame({
              '$|\Delta m^R-\Delta m|$': df_means_r.error_gap.round(decimals=4).astype(str) + '(\pm' + df_std_r.error_gap.round(4).astype(str) + ')',
              'Chebyshev Bound (R)': df_means_r.Chebyshev.round(4).astype(str),
              '$|\Delta m^L-\Delta m|$': df_means_l.error_gap.round(4).astype(str) + '(\pm' + df_std_l.error_gap.round(4).astype(str) + ')',
              'Chebyshev Bound (L)': df_means_l.Chebyshev.round(4).astype(str),
             })
errors

,$|\Delta m^R-\Delta m|$,Chebyshev Bound (R),$|\Delta m^L-\Delta m|$,Chebyshev Bound (L)
e,,,,
0.01,0.0739(\pm0.0717),1.2586,0.0438(\pm0.0488),1.0525
0.10,0.0117(\pm0.0074),0.1206,0.0082(\pm0.0043),0.106
1.00,0.0005(\pm0.0003),0.0094,0.001(\pm0.0008),0.0118
1.78,0.0003(\pm0.0003),0.0052,0.0004(\pm0.0004),0.0071
10.00,0.0001(\pm0.0001),0.0032,0.0001(\pm0.0),0.0009


In [24]:
print(errors.drop(1.78).to_latex(index=True, escape=False))

\begin{tabular}{lllll}
\toprule
{} & $|\Delta m^R-\Delta m|$ & Chebyshev Bound (R) & $|\Delta m^L-\Delta m|$ & Chebyshev Bound (L) \\
e     &                         &                     &                         &                     \\
\midrule
0.01  &       0.0739(\pm0.0717) &              1.2586 &       0.0438(\pm0.0488) &              1.0525 \\
0.10  &       0.0117(\pm0.0074) &              0.1206 &       0.0082(\pm0.0043) &               0.106 \\
1.00  &       0.0005(\pm0.0003) &              0.0094 &        0.001(\pm0.0008) &              0.0118 \\
10.00 &       0.0001(\pm0.0001) &              0.0032 &          0.0001(\pm0.0) &              0.0009 \\
\bottomrule
\end{tabular}

